In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Load the data
digits = np.load('unlabelled_train_data_images.npy')
X = digits.reshape(digits.shape[0], -1) / 255.0  # Flatten and normalize

# 1. Create a more powerful CNN feature extractor (using transfer learning concepts)
def create_deep_cnn_embeddings(X, encoding_dim=64):
    """
    Create embeddings using a deeper CNN with residual connections
    """
    # Reshape for CNN
    input_shape = (28, 28, 1)
    X_reshaped = X.reshape(-1, 28, 28, 1)
    
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)
    
    # First convolutional block
    x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Second convolutional block
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Feature extraction
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    encoded = layers.Dense(encoding_dim, activation='relu')(x)
    
    # Create model
    feature_extractor = models.Model(inputs=inputs, outputs=encoded)
    
    # Create a simple autencoder (optional - for pretraining)
    decoded = layers.Dense(784, activation='sigmoid')(encoded)
    autoencoder = models.Model(inputs=inputs, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    # Pretrain the autoencoder to get better feature representations
    print("Pretraining feature extractor using autoencoder...")
    autoencoder.fit(X_reshaped, X, 
                   epochs=5, 
                   batch_size=256,
                   shuffle=True,
                   validation_split=0.1,
                   verbose=1)
    
    # Get embeddings using the encoder part
    embeddings = feature_extractor.predict(X_reshaped)
    print(f"CNN embedding shape: {embeddings.shape}")
    
    return embeddings, feature_extractor

# 2. Improved clustering with hyperparameter tuning and stability checking
def improved_clustering(embeddings, min_clusters=8, max_clusters=12):
    """
    Find the optimal number of clusters and perform stabilized clustering
    """
    # Try different numbers of clusters and evaluate using silhouette score
    scores = []
    models = []
    
    print("Finding optimal number of clusters...")
    for n_clusters in range(min_clusters, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        scores.append(score)
        models.append(kmeans)
        print(f"  {n_clusters} clusters: silhouette score = {score:.4f}")
    
    # Choose optimal number of clusters
    best_idx = np.argmax(scores)
    best_n_clusters = min_clusters + best_idx
    best_model = models[best_idx]
    
    print(f"Optimal number of clusters: {best_n_clusters} (score: {scores[best_idx]:.4f})")
    
    # Perform multiple clusterings and ensemble the results for stability
    print("Performing ensemble clustering for stability...")
    ensemble_models = []
    ensemble_labels = []
    
    # Create 5 clusterings with different random states
    for i in range(5):
        kmeans = KMeans(n_clusters=best_n_clusters, random_state=i*10, n_init=10)
        labels = kmeans.fit_predict(embeddings)
        ensemble_models.append(kmeans)
        ensemble_labels.append(labels)
    
    # Create a consensus clustering by selecting the most common label
    # Note: This requires solving the label correspondence problem which is complex
    # For simplicity, we'll use the best single clustering instead
    labels = best_model.fit_predict(embeddings)
    
    return labels, best_model, best_n_clusters

# 3. Visualize results with better dimensionality reduction
def visualize_clusters(embeddings, labels, n_clusters):
    """
    Visualize clusters with improved t-SNE and evaluate cluster quality
    """
    # Use PCA first to reduce dimensionality before t-SNE (better results)
    if embeddings.shape[1] > 50:
        print("Reducing dimensions with PCA before t-SNE...")
        pca = PCA(n_components=50)
        embeddings_reduced = pca.fit_transform(embeddings)
    else:
        embeddings_reduced = embeddings
    
    # Use t-SNE with better parameters
    print("Running t-SNE for visualization...")
    tsne = TSNE(n_components=2, 
                perplexity=50, 
                learning_rate='auto',
                n_iter=1000, 
                random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings_reduced)
    
    # Plot results with improved visuals
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                          c=labels, cmap='tab10', alpha=0.7, s=5)
    plt.colorbar(scatter)
    plt.title('t-SNE visualization of digit clusters', fontsize=15)
    plt.tight_layout()
    plt.show()
    
    # Plot representative samples from each cluster
    plot_cluster_examples(digits, labels, n_clusters)
    
    return embeddings_2d

# 4. Plot examples from each cluster with better organization
def plot_cluster_examples(images, labels, n_clusters, samples_per_cluster=10):
    """
    Display samples from each cluster in an organized grid
    """
    plt.figure(figsize=(samples_per_cluster * 1.2, n_clusters * 1.2))
    
    # For each cluster
    for cluster_id in range(n_clusters):
        # Get indices of images in this cluster
        cluster_indices = np.where(labels == cluster_id)[0]
        
        # Skip if cluster is empty
        if len(cluster_indices) == 0:
            continue
        
        # Take a sample of images from this cluster
        sample_size = min(samples_per_cluster, len(cluster_indices))
        sample_indices = np.random.choice(cluster_indices, sample_size, replace=False)
        
        # Display sample images
        for j, idx in enumerate(sample_indices):
            plt.subplot(n_clusters, samples_per_cluster, cluster_id * samples_per_cluster + j + 1)
            plt.imshow(images[idx].reshape(28, 28), cmap='gray')
            plt.axis('off')
            
            # Label the first image in each row
            if j == 0:
                plt.title(f'Cluster {cluster_id}', fontsize=10)
    
    plt.suptitle("Samples from each cluster", y=0.95, fontsize=16)
    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    plt.show()

# 5. Evaluate consistency across multiple runs and identify representative digits
def evaluate_cluster_stability(digits, labels, n_clusters):
    """
    Analyze clusters to find their likely digit identities and evaluate consistency
    """
    # Find the centroid images for each cluster
    centroids = []
    for i in range(n_clusters):
        cluster_indices = np.where(labels == i)[0]
        if len(cluster_indices) > 0:
            cluster_samples = digits[cluster_indices].reshape(len(cluster_indices), -1)
            centroid = np.mean(cluster_samples, axis=0)
            # Find the image closest to the centroid
            distances = np.linalg.norm(cluster_samples - centroid, axis=1)
            representative_idx = cluster_indices[np.argmin(distances)]
            centroids.append((i, representative_idx))
    
    # Display centroids with higher resolution
    plt.figure(figsize=(15, 4))
    for i, (cluster_id, idx) in enumerate(centroids):
        plt.subplot(2, 5, i+1)
        plt.imshow(digits[idx].reshape(28, 28), cmap='gray', interpolation='none')
        plt.title(f"Cluster {cluster_id}")
        plt.axis('off')
    plt.tight_layout()
    plt.suptitle("Representative examples from each cluster", y=1.02, fontsize=16)
    plt.show()
    
    print("Please manually assign digit labels to each cluster based on the representative images.")
    print("Example: cluster_to_digit = {0:3, 1:4, 2:1, ...}")
    
    return centroids

# Main execution
# 1. Generate improved embeddings
deep_embeddings, deep_model = create_deep_cnn_embeddings(X)

# 2. Perform improved clustering
cluster_labels, cluster_model, n_clusters = improved_clustering(deep_embeddings)

# 3. Visualize results
tsne_embedding = visualize_clusters(deep_embeddings, cluster_labels, n_clusters)

# 4. Evaluate and identify clusters
centroids = evaluate_cluster_stability(digits, cluster_labels, n_clusters)

# 5. Save results
np.save('improved_digit_labels.npy', cluster_labels)
np.savez('improved_clustering_results.npz', 
         labels=cluster_labels, 
         embeddings=deep_embeddings,
         tsne=tsne_embedding)

print("Improved clustering complete! The new labels have been saved.")